# Equal-Weight S&P 500 Index Fund
    
## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import math 
import requests
import pandas as pd
import numpy as np
import xlsxwriter

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
497    YUM
498    ZBH
499   ZBRA
500   ZION
501    ZTS

[502 rows x 1 columns]


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing  API Call

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

In [6]:
my_columns = ['Ticker','Stock Price','Market Capitalization','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
print(final_dataframe)

Empty DataFrame
Columns: [Ticker, Stock Price, Market Capitalization, Number of Shares to Buy]
Index: []


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,172.6,2931768480528,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:10]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    print(data)
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
            index = my_columns),
        ignore_index = True
    )

{'avgTotalVolume': 2471524, 'calculationPrice': 'tops', 'change': -4.85, 'changePercent': -0.03537, 'close': 0, 'closeSource': 'olciiaff', 'closeTime': None, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': ' re ladc1tee5y iuenpidm', 'highTime': 1669713394633, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 140.26, 'iexBidSize': 100, 'iexClose': 136.77, 'iexCloseTime': 1683968309739, 'iexLastUpdated': 1694487974514, 'iexMarketPercent': 0.06082351561557742, 'iexOpen': 141.8, 'iexOpenTime': 1709458564812, 'iexRealtimePrice': 139.34, 'iexRealtimeSize': 36, 'iexVolume': 39477, 'lastTradeTime': 1718732168499, 'latestPrice': 136.75, 'latestSource': 'IEX real time price', 'latestTime': '2:31:01 PM', 'latestUpdate': 1714180313196, 'latestVolume': None, 'low': 0, 'lowSource': 'tec1dmnye u darie ei5pl', '

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.750,41127173230,N/A
1,AAL,16.970,10593448345,N/A
2,AAP,218.690,13129510598,N/A
3,AAPL,173.600,2910665455714,N/A
4,ABBV,160.870,284067966855,N/A
5,ABC,157.490,32799255950,N/A
6,ABMD,323.800,14605894517,N/A
7,ABT,119.687,212087858035,N/A
8,ACN,329.240,217942652622,N/A
9,ADBE,419.980,206285711497,N/A


## Using Batch API Calls to Improve Performance

In [10]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [11]:
symbol_groups  = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data.status_code)
    print(batch_api_call_url)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.22,41863653506,N/A
1,AAL,16.83,11015330732,N/A
2,AAP,216.20,13556815698,N/A
3,AAPL,178.90,2880783312895,N/A
4,ABBV,162.37,290799825867,N/A
...,...,...,...,...
497,YUM,120.19,34023013682,N/A
498,ZBH,125.79,26261510440,N/A
499,ZBRA,428.64,22886658497,N/A
500,ZION,67.94,10816222936,N/A


## Calculating the Number of Shares to Buy

In [12]:
# portfolio_size = input('Enter the value of your portfolio')
portfolio_size = 10000
try:
    val = float(portfolio_size)
except ValueError:
    print("that is not number!")
    portfolio_size = input('Enter the value of your portfolio')
    val = float(porfolio_size)

In [13]:
position_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.22,41863653506,0
1,AAL,16.83,11015330732,1
2,AAP,216.20,13556815698,0
3,AAPL,178.90,2880783312895,0
4,ABBV,162.37,290799825867,0
...,...,...,...,...
497,YUM,120.19,34023013682,0
498,ZBH,125.79,26261510440,0
499,ZBRA,428.64,22886658497,0
500,ZION,67.94,10816222936,0


## Formatting Our Excel Output

### Initializing  XlsxWriter Object

In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx',engine = 'xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index= False)

### Creating the Formats We'll Need For Our `.xlsx` File

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border' : 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

In [16]:

writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization',dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy',integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [17]:
column_formats = {
    'A' : ['Ticker',string_format],
    'B' : ['Stock Price',dollar_format],
    'C' : ['Market Capitalization',dollar_format],
    'D' : ['Number of Shares to Buy',integer_format]
 }
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy:

In [18]:
writer.save()

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
